### Install required libraries

In [1]:
# %pip install -r requirements.txt

### Prepare data

In [2]:
import numpy as np
from utils import onehot_encode


def stratified_split(val_ratio, data):
    classes = np.unique(data['target'])

    train_indices = []
    val_indices = []

    for cls in classes:

        cls_indices = np.where(data['target'] == cls)[0]

        # random state for reproducibility
        rng = np.random.RandomState(seed=1318)
        cls_indices = rng.permutation(cls_indices)        
        val_size = int(len(cls_indices) * val_ratio)
        val_cls_indices = cls_indices[:val_size]
        train_cls_indices = cls_indices[val_size:]
        
        val_indices.extend(val_cls_indices)
        train_indices.extend(train_cls_indices)

    
    train_split = data[train_indices]
    val_split = data[val_indices]
    return train_split, val_split

def z_score_normmalize(x):
    mean = np.mean(x, axis=0)
    std = np.std(x, axis=0)
    x = (x - mean) / std
    return x

data = np.loadtxt(
    'dataset/2d.trn.dat', 
    skiprows=1,
    dtype=[('x', float), ('y', float), ('target', 'U1')]
)

test_data = np.loadtxt(
    'dataset/2d.tst.dat', 
    skiprows=1,
    dtype=[('x', float), ('y', float), ('target', 'U1')]
)

train_data, val_data = stratified_split(0.2, data)

X_train = z_score_normmalize(np.column_stack((train_data['x'], train_data['y'])))
y_train = onehot_encode(train_data['target'])

X_val = z_score_normmalize(np.column_stack((val_data['x'], val_data['y'])))
y_val = onehot_encode(val_data['target'])

X_test = z_score_normmalize(np.column_stack((test_data['x'], test_data['y'])))
y_test = onehot_encode(test_data['target'])

print(f"X_train shape {X_train.shape}")
print(f"X_val shape {X_val.shape}")
print(f"X_test shape {X_test.shape}")

X_train shape (6401, 2)
X_val shape (1599, 2)
X_test shape (2000, 2)


## Train Neural Network

In [ ]:
from neural_network import NeuralNetwork, Layer, ActivationFunction
from utils import compute_accuracy


nn = NeuralNetwork(
    layers=[
        Layer(ActivationFunction.relu, 12, input_dim=2),
        Layer(ActivationFunction.relu, 8),
        Layer(ActivationFunction.softmax, 3),
    ]
)

    
train_losses, val_losses = nn.train(X_train.T, y_train.T, lr=0.05, epochs=1000, X_val=X_val.T, y_val=y_val.T)
pred = nn.predict(X_test)
print(compute_accuracy(y_test.T, pred))

In [ ]:
from matplotlib import pyplot as plt

plt.subplot(1, 1, 1)
plt.plot(train_losses,label='train')
plt.plot(val_losses, label='val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epoch')
plt.legend()

In [ ]:
print(len(train_losses))
print(len(val_losses))

1000
0
